In [ ]:
from marxslynx.lynx import PerfectLynx, conf

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.utils import generate_test_photons
from marxs.simulator import Sequence

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
conf['grating_size'] = np.array([150., 150.])

In [ ]:
n_photons = 1e4
instrument = PerfectLynx(conf)

In [ ]:
len(instrument.elements[2].elements)

In [ ]:
mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy=0.5,
                           flux=1.)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(n_photons)
photons = fixedpointing(photons)
photons = Sequence(elements=instrument.elements[:2])(photons)
p = Sequence(elements=instrument.elements[2:])(photons.copy())

    

In [ ]:
out = plt.hist(p['detcirc_phi'], weights=p['probability'], bins=100)

In [ ]:
instrument.elements

In [ ]:
from marxs.optics import OrderSelector
from marxs.math.utils import e2h, h2e, norm_vector

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
class ChirpFinder():
    u = 0
    v = 0
    energy = 0.5
    order = 0
    def __init__(self, detector, grat, order):
        self.photon = generate_test_photons(1)
        self.detector = detector
        self.grat = grat
        self.set_order(order)
    def set_energy(self, energy):
        self.energy = energy
        self.set_order(self.order)
    def set_uv(self, u, v):
        self.u = u
        self.v = v
        self.init_photon()
    def set_order(self, order):
        self.order = order
        self.grat.order_selector = OrderSelector([order])
        self.grat._d = 0.002
        self.set_uv(0., 0.)
        self.run_photon()
        self.goal = self.photon['detcirc_phi'][0]
    def init_photon(self):
        posongrat = h2e(self.grat.geometry['center'] + self.u  * self.grat.geometry['v_y'] + self.v * self.grat.geometry['v_z'])
        self.pos = e2h(1.1 * posongrat, 1)
        self.dir = norm_vector(- e2h(posongrat.reshape(1, 3), 0))
        self.reset_photon()
    def reset_photon(self):
        self.photon['pos'] = self.pos
        self.photon['dir'] = self.dir
    def run_photon(self):
        self.reset_photon()
        self.photon = self.grat(self.photon)
        self.photon = self.detector(self.photon)
    def optimize_func(self, d):
        self.grat._d = d * 0.002
        self.run_photon()
        return np.abs(self.photon['detcirc_phi'][0] - self.goal)
        
    


In [ ]:
opt = ChirpFinder(instrument.elements[9], instrument.elements[2].elements[0], order=-5)

In [ ]:
res = np.zeros((len(instrument.elements[2].elements), 3, 3))
for i, grat in enumerate(instrument.elements[2].elements):
    if np.mod(i, 10) == 0:
        print(i)
    opt.grat = grat
    for j, u in enumerate(np.linspace(-.99, .99, res.shape[1])):
        #for k, v in enumerate(np.linspace(-.99, .99, res.shape[2])):
        for k, v in enumerate([0]):
            opt.set_uv(u, v)
            res[i, j, k] = minimize_scalar(opt.optimize_func, bracket=(.99, 1., 1.01)).x

In [ ]:
epos = np.stack(instrument.elements[2].elem_pos)
epos.shape

In [ ]:
epos[300, :, 3]

In [ ]:
res.shape

In [ ]:
plt.scatter(epos[:, 1, 3], epos[:, 2, 3], c = res[:, 0, 1]-res[:, 2, 1])
plt.colorbar()

In [ ]:
res.shape

In [ ]:
from scipy.interpolate import RectBivariateSpline

In [ ]:
dbetter = res
#dbetter = np.ones_like(res)
instrumchirp = PerfectLynx(conf)
for i, grat in enumerate(instrumchirp.elements[2].elements):
    ly = np.linalg.norm(grat.geometry['v_y'])
    lz = np.linalg.norm(grat.geometry['v_z'])
    grat.spline = RectBivariateSpline(
                        ly * np.linspace(-.99, .99, res.shape[1]), 
                        lz * np.linspace(-.99, .99, res.shape[1]), 
                        0.0002 * dbetter[0, :, :],
                        bbox=[-ly, ly, -lz, lz],
                        kx=2, ky=2)
    def func(intercoos):
        #print(intercoos, grat.spline(intercoos[:, 0], intercoos[:, 1], grid=False))
        return grat.spline(intercoos[:, 0], intercoos[:, 1], grid=False)
    grat._d = func

In [ ]:
ptest = Sequence(elements=instrumchirp.elements[2:])(photons[:100].copy())


In [ ]:
chirprunner = Sequence(elements=instrumchirp.elements[2:])
pchirp = chirprunner(photons.copy())

In [ ]:
chirprunner.elements[0].elements[0].geometry['v_z']

In [ ]:
np.linalg.norm(grat.geometry['v_z']), np.linalg.norm(grat.geometry['v_y'])

In [ ]:
bins = np.linspace(.12285, .12288, 40)
out = plt.hist(p['detcirc_phi'], weights=p['probability'], bins=bins)
out = plt.hist(pchirp['detcirc_phi'], weights=pchirp['probability'], bins=bins, histtype='step')

In [ ]:
ind = p['order'] == -5
plt.plot(p['detcirc_phi'][ind], p['detcirc_y'][ind], '.')
ind = pchirp['order'] == -5
plt.plot(pchirp['detcirc_phi'][ind], pchirp['detcirc_y'][ind], '.')
#plt.xlim([.12284, .1229])
#plt.ylim([-0.2, 0.2])

In [ ]:
bins = np.linspace(-0.0014, -0.0009, 50)
ind = p['order'] == -5
out = plt.hist(p['detcirc_phi'][ind], weights=p['probability'][ind], bins=bins, label='p')
print(np.std(p['detcirc_phi'][ind]))
ind = pchirp['order'] == -5
out = plt.hist(pchirp['detcirc_phi'][ind], weights=pchirp['probability'][ind], bins=bins, histtype='step', label='chirp')
print(np.std(pchirp['detcirc_phi'][ind]))
plt.legend()

In [ ]:
bins = np.linspace(-0.0014, -0.0009, 50) + 0.00125 + 0.1229 
ind = p['order'] == -0
out = plt.hist(p['detcirc_phi'][ind], weights=p['probability'][ind], bins=bins)
print(np.std(p['detcirc_phi'][ind]))
ind = pchirp['order'] == -0
out = plt.hist(pchirp['detcirc_phi'][ind], weights=pchirp['probability'][ind], bins=bins, histtype='step')
print(np.std(pchirp['detcirc_phi'][ind]))

In [ ]:
out[1].mean()

In [ ]:
from marxs.analysis.gratings import resolvingpower_from_photonlist

In [ ]:
res0, pos0, std0 = resolvingpower_from_photonlist(p, np.arange(-8, 1), col='detcirc_phi')

In [ ]:
resc, posc, stdc = resolvingpower_from_photonlist(pchirp, np.arange(-8, 1), col='detcirc_phi')

In [ ]:
plt.plot(np.arange(-8, 1), res0)
plt.plot(np.arange(-8, 1), resc)

In [ ]:
go = instrument.elements[2].elements[96]
gc = instrumchirp.elements[2].elements[96]

In [ ]:
gc.pos4d[:,3]

In [ ]:
uv = np.mgrid[-.99: .99:0.01, -.99: .99:0.01]

In [ ]:
uv[0].flatten()

In [ ]:
from marxs.utils import generate_test_photons
from marxs.math.utils import h2e, e2h, norm_vector
phot_in = generate_test_photons(uv[0].flatten().shape[0])
phot_in['energy'] = 0.5
phot_in['pos'] = go.geometry['center'][None, :] + 100 * go.geometry['e_x'][None, :] + uv[0].flatten()[:, None] * go.geometry['v_y'][None, :] + uv[1].flatten()[:, None] * go.geometry['v_z'][None, :]
phot_in['dir'] = -norm_vector(e2h(h2e(phot_in['pos']), 0))

In [ ]:
instrument.elements

In [ ]:
p1 = Sequence(elements=[go, instrument.elements[9]])(phot_in.copy())
p2 = Sequence(elements=[gc, instrument.elements[9]])(phot_in.copy())

In [ ]:
from marxs.optics import OrderSelector
go.order_selector = OrderSelector([-5])
gc.order_selector = go.order_selector

In [ ]:
ind = p1['detcirc_phi'] < 0.12
plt.plot(p1['detcirc_phi'][ind], p1['detcirc_y'][ind], '.')
plt.plot(p2['detcirc_phi'][ind], p2['detcirc_y'][ind], '.')

In [ ]:
bins = 100
out = plt.hist(p1['detcirc_phi'][ind], weights=p1['probability'][ind], bins=bins)
out = plt.hist(p2['detcirc_phi'][ind], weights=p2['probability'][ind], bins=bins, histtype='step')

In [ ]:
ind

In [ ]:
plt.plot(p1['grat_y'], p1['grat_z'], '.')

In [ ]:
go._d

In [ ]:
gc._d()

In [ ]:
gc._d(np.vstack([p2['grat_y'].data, p2['grat_z'].data]).T)